In [81]:
import sys
from time import sleep
sys.path.append('..')

In [82]:
from pprint import pprint
import time

In [83]:
from RaAuto.Pmo import OpenPMO, HandleInternalMail
from RaAuto.RaAuto import *
from RaAuto.RaTime import add_minutes

In [84]:
import pyautogui as pag

In [85]:
class ImgsEnum:
    def __init__(self, imagesPath):
        self.TenMinuteMark = os.path.join(imagesPath, "Schedule", "TenMinuteMark.png")
        self.AndraMallTillfalligt = os.path.join(imagesPath, "Schedule", "AndraMallTillfalligt.png")
        self.IdagButton = os.path.join(imagesPath, "IdagButton.png")
        self.ResurserLabel = os.path.join(imagesPath, "Schedule", "ResurserLabel.png")
        self.StaffMemberCheckboxChecked = os.path.join(imagesPath, "Schedule", "StaffMemberCheckboxChecked.png")
        self.StaffMemberCheckboxCheckedOnUnselectedRow = os.path.join(imagesPath, "Schedule", "StaffMemberCheckboxCheckedOnUnselectedRow.png")
        self.NyBokningTitle = os.path.join(imagesPath, "NyBokningTitle.png")
        self.WarningSymbolDoubleBookingLarge = os.path.join(imagesPath, "WarningSymbolDoubleBookingLarge.png")
        self.AvbrytMenuButton = os.path.join(imagesPath, "AvbrytMenuButton.png")
        self.VardgarantiCheckboxChecked = os.path.join(imagesPath, "VardgarantiCheckboxChecked.png")


IMG = ImgsEnum(os.path.join("..", "Images"))

In [86]:
def GoToUserSelectionArea():
    HandleInternalMail()
    pos = waitForElement(IMG.ResurserLabel)
    if (pos):
        x = pos.x + 50
        y = pos.y + 100
        return Pos(x, y)
    return False


In [87]:
def ScrollUp():
    sleep(0.2)
    pag.scroll(500)
    sleep(0.2)
    pag.scroll(500)
    sleep(0.2)

In [88]:
def DeselectAllUsers(repeats=5):
    pos = GoToUserSelectionArea()

    if not pos:
        return False
    
    pag.moveTo(pos.x, pos.y)
    ScrollUp()

    for i in range(repeats):
        for j in range(3):
            waitAndClickElement(IMG.StaffMemberCheckboxChecked, beforeStart=0.1, limit=0.1, waitTime=0.1, warn=False)
        pag.moveTo(pos.x, pos.y)
        pag.scroll(-250)
        sleep(0.2)

    if HandleInternalMail():
        pos = DeselectAllUsers(repeats=5)
        
    return pos


# pos = OpenPMO()
# if pos:
#     DeselectAllUsers()

In [89]:
def DeselectSingleUser():
    return waitAndClickElement(IMG.StaffMemberCheckboxChecked, beforeStart=0.1, limit=0.1, waitTime=0.1, warn=False)

In [90]:
def CheckIfUsesIsSelectedCorrectly(userName):
    pos = waitForElement(IMG.IdagButton)
    if pos:
        text = capture_and_ocr(pos.x, pos.y + 13, width=500)
        print(f'Text: "{text}"')
        if userName in text:
            print("User correctly selected")
            return pos
        else:
            print("USER NOT CORRECTLY SELECTED!")
            return False

In [91]:
def SetUser(userName):
    # DeselectSingleUser()

    pos = GoToUserSelectionArea()

    if not pos:
        return False
    
    pag.moveTo(pos.x, pos.y)
    # ScrollUp()
    pag.leftClick()

    # words = userName.split(" ")
    waitAndType(userName)
    waitAndPress("space")
    pos = CheckIfUsesIsSelectedCorrectly(userName)
    print(f"User was {'selected correctly' if pos else 'not selected correctly'}")
    if pos:
        return pos
    else:
        if " " in userName:
            waitAndClickElement(IMG.StaffMemberCheckboxCheckedOnUnselectedRow, relX=9)
    
    pos = CheckIfUsesIsSelectedCorrectly(userName)
    if pos:
        return pos
    else:
        raise RuntimeError("Could not select user {userName} correctly")
    

# pos = OpenPMO()
# if pos:
#     DeselectAllUsers()
#     SetUser("Linda Y")


In [92]:
def SetDate(date):
    pos = waitAndClickElement(IMG.IdagButton, relX=-60)
    if pos:
        waitAndPress("ctrl", "a")
        waitAndType(date)
        if HandleInternalMail():
            pos = SetDate(date)
        else:
            waitAndPress("enter")
    return pos
    

# pos = OpenPMO()
# if pos:
#     SetDate("2024-06-03")

In [93]:
# negativePreviousCount is because the dialogue may already have a count set higher than 1
def SetTimeSlots(typeStr, startTime, count, negativePreviousCount=0):
    pos = waitAndClickElement(IMG.TenMinuteMark, relX=15, button="right")
    if pos:
        pos = waitAndPress("m")
    if pos:
        pos = waitForElement(IMG.AndraMallTillfalligt)
    if pos:
        waitAndType(typeStr)
        waitAndPress("tab")
        waitAndType(startTime)
        sleep(0.05)
        waitAndPress("tab")
        waitAndPress("tab")
        if (negativePreviousCount > 0):
            sleep(0.4)
            for i in range(negativePreviousCount):
                waitAndPress("down", beforeStart=0.07)
        sleep(0.4)
        for i in range (1, count):
            waitAndPress("up", beforeStart=0.07)
        sleep(0.05)
        if HandleInternalMail():
            waitAndPress("alt", "o")
            pos = SetTimeSlots(typeStr, startTime, count, negativePreviousCount)
        else:
            waitAndPress("alt", "o")
            pos = waitForElementDisappear(IMG.AndraMallTillfalligt)
            sleep(0.1)

    return pos


# pos = OpenPMO()
# if pos:
#     SetTimeSlots("ad", "08:00", 30, 30)

In [94]:
def AddBooking(comment, type, startTime, minutes):

    pos = waitAndClickElement(IMG.TenMinuteMark, relX=12)
    waitAndPress("alt", "n")
    pos = waitForElement(IMG.NyBokningTitle)

    if pos:
        for i in range(9):
            waitAndPress("tab", beforeStart=0.025)
        waitAndType(type)
        waitAndPress("tab")
        waitAndPress("tab")
        waitAndType(startTime)
        waitAndPress("tab")
        waitAndType(str(minutes))
        waitAndPress("tab")
        waitAndPress("tab")
        if HandleInternalMail():
            waitAndClickElement(IMG.AvbrytMenuButton, limit=1)
            waitForElementDisappear(IMG.NyBokningTitle)
            sleep(0.5)
            pos = AddBooking(comment, type, startTime, minutes)
        else:
            waitAndType(comment)
            waitAndClickElement(IMG.VardgarantiCheckboxChecked, relX=-7, limit=0.2, conf=0.95, warn=False)

            if HandleInternalMail():
                waitAndClickElement(IMG.AvbrytMenuButton, limit=1)
                waitForElementDisappear(IMG.NyBokningTitle)
                sleep(0.5)
                pos = AddBooking(comment, type, startTime, minutes)
            else:
                waitAndPress("alt", "s")
                tempPos = waitForElement(IMG.WarningSymbolDoubleBookingLarge, limit=1.5, waitTime=0.5, warn=False)
                if tempPos:
                    waitAndPress("enter")
            waitForElementDisappear(IMG.NyBokningTitle)
            sleep(0.5)

    return pos

# pos = OpenPMO()
# if pos:
#     AddBooking("Special2", "ad", "10:00", 40)



# Import and filters

In [95]:
from datetime import datetime
import json

In [96]:
with open('Input/schedule.json') as file:
    scheduleData = json.load(file)

staff_dict = scheduleData["staffMemberDict"]
room_dict = scheduleData["roomDict"]
staff_schedules = scheduleData["schedule"]

In [97]:
staff_dict

{'1': 'Rasmus',
 '2': 'Linn',
 '3': 'Maria MC',
 '4': 'Maria K',
 '5': 'Dhay',
 '6': 'Michael',
 '7': 'Julijana',
 '8': 'Anna',
 '9': 'Linda Y',
 '10': 'Isac'}

In [98]:
room_dict

{'1': 'ST rum 1',
 '2': 'ST rum 2',
 '3': 'LW rum',
 '4': 'JK rum',
 '5': 'MT rum',
 '6': 'Lokalen',
 '7': 'Hemifrån'}

In [99]:
activityDict ={
    "ns": "<",
    "ad": "ad",
    "em": "ak",
    "pl": "mo",
    "ph": "tel",
    "br": "ra",
    "me": "ex",
    "sp": "sp",
    "mi": "öv",
    "ch": "läkartid b",
}

### Structure of the schedule.json file as described in typescript

<pre>
interface staffMemberScheduleDay {
    staffPk: number;
    personDays: SchedulePersonDayType[];
}

type ScheduleByStaffMemberList = staffMemberScheduleDay[];

interface exportJson {
    staffMemberDict: { [staffPk: number]: string };
    schedule: ScheduleByStaffMemberList;
}

interface SchedulePersonDayType {
    pk: number;
    staff_member: number;
    date: string;
    schedule_chunk: ScheduleChunkType;
    is_dirty?: boolean;
    is_saving?: boolean;
}

interface ScheduleChunkType {
    pk: number;
    start_time: string;
    time_resolution: 5 | 10 | 15 | 30 | 60;
    time_slots: TimeSlotType[];
    last_saved: string;
    last_saved_by: number | null;
}

interface TimeSlotType {
    a: ActivityString; // Activity
    c?: string; // Comment
    r?: string; // Room
}
</pre>

## FILTER

In [100]:
#staff_schedules[0]["personDays"][0]

# Filters:
start_date = None # "2024-07-01"
end_date = None # "2024-07-13"
staff_members = [10]#[1, 2, 3, 4, 5, 8, 9, 10]

print(len(staff_schedules))
filtered_staff_schedules = staff_schedules
if staff_members:
    filtered_staff_schedules = [schedule for schedule in staff_schedules if schedule["staffPk"] in staff_members]
print(len(filtered_staff_schedules))

8
1


In [101]:
if isinstance(start_date, str):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
if isinstance(end_date, str):
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

print(len(filtered_staff_schedules[0]["personDays"]))
if start_date and end_date:
    filtered_staff_schedules = [
        {
            "staffPk": schedule["staffPk"],
            "personDays": [
                person_day for person_day in schedule["personDays"]
                if start_date <= datetime.strptime(person_day["date"], "%Y-%m-%d") <= end_date
            ]
        }
        for schedule in filtered_staff_schedules
    ]

print(len(filtered_staff_schedules[0]["personDays"]))

28
28


In [105]:
# example_chunk = staff_schedules[0]["personDays"][0]["schedule_chunk"]
# example_chunk

In [106]:
def SetScheduleChunk(chunk):
    start_time = chunk["start_time"][:5]
    time_resolution = chunk["time_resolution"]
    time_slots = chunk["time_slots"]

    index = 0
    prev_count = 30

    while index < len(time_slots):
        activity_start_time = add_minutes(start_time, index * time_resolution)
        activity = time_slots[index]["a"]
        comment = time_slots[index].get("c", "")
        room = time_slots[index].get("r", "")
        room_str = room_dict.get(room, "")
        activity_str = activityDict[activity]

        activity_count = 0
        comment_count = 0
        
        for i in range(index, len(time_slots)):
            if time_slots[i]["a"] == activity and time_slots[i].get("c", "") == comment:
                activity_count += 1
                comment_count += 1

                if activity_count == 30:
                    break   # Max for what PMO can handle
            else:
                break

        comment_minutes = comment_count * time_resolution

        if activity != "ns":
            pos = SetTimeSlots(activity_str, activity_start_time, activity_count, prev_count)
            prev_count = activity_count + 5
        else:
            pos = True
        
        if pos and comment:
            AddBooking(comment, activity_str, activity_start_time, comment_minutes)

        if pos and room:
            AddBooking(room_str, activity_str, activity_start_time, time_resolution)

        index += activity_count

        if not pos:
            break

    if activity != "ns":
        SetTimeSlots(activityDict["ns"], add_minutes(start_time, time_resolution * len(time_slots)), 6, prev_count)
    
    return pos


# pos = OpenPMO()
# if pos:
#     SetScheduleChunk(example_chunk)

# Actually run

In [107]:
continue_from_user = None # 8
continue_from_date = "2024-06-18"

skip_users = False
skip_dates = False
if continue_from_user:
    skip_users = True
if continue_from_date:
    skip_dates = True

OpenPMO()
pos = DeselectAllUsers()

total_time = 0
completed_users = 0

if pos:
    for staff_schedule in filtered_staff_schedules:
        userId = staff_schedule["staffPk"]
        user = staff_dict[str(userId)]
        if skip_users:
            if userId == continue_from_user:
                skip_users = False
            else:
                print(f"Skipping {user}")
                continue

        print(f"""
###########################################
              # {user} #
###########################################""")
        
        start_time = time.time()
        
        DeselectSingleUser()
        pos = SetUser(user)

        if pos:
            for person_day in staff_schedule["personDays"]:
                date_str = person_day["date"]

                if skip_dates:
                    if date_str == continue_from_date:
                        skip_dates = False
                    else:
                        print(f"Skipping {date_str}")
                        continue

                print(date_str)
                pos = SetDate(date_str)
                if not pos:
                    break

                chunk = person_day["schedule_chunk"]
                # chunk_start_time = chunk["start_time"]
                # time_resolution = chunk["time_resolution"]
                # time_slots = chunk["time_slots"]
                
                pos = SetScheduleChunk(chunk)
                print("Completed day")
                if not pos:
                    break

        end_time = time.time()
        user_time = end_time - start_time
        total_time += user_time
        completed_users += 1

        print(f"\nCompleted {user} in {user_time / 60:.1f} minutes\n")

average_time = total_time / completed_users if completed_users > 0 else 0
print(f"Total time: {total_time} seconds, Average time per user: {average_time / 60:.1f} minutes")


###########################################
              # Isac #
###########################################
Text: "Lund : Isac Wickmnan"
User correctly selected
User was selected correctly
2024-06-03
Completed day
2024-06-04
Completed day
2024-06-05
Completed day
2024-06-06
Completed day
2024-06-07
Completed day
2024-06-08
Completed day
2024-06-09
Completed day
2024-06-10
Completed day
2024-06-11
Completed day
2024-06-12
Completed day
2024-06-13
Completed day
2024-06-14
Completed day
2024-06-15
Completed day
2024-06-16
Completed day
2024-06-17
Completed day
2024-06-18
Completed day

Completed Isac in 50.5 minutes

Total time: 3030.804972887039 seconds, Average time per user: 50.5 minutes
